# Chapter 25: Linting, Formatting, and Testing

Automated tooling for code quality and testing is essential for professional Python
development. This notebook covers linting with `ruff`, auto-formatting with `black`,
import sorting, and comprehensive testing with `pytest`.

## Topics Covered
- **Linting**: `ruff`, `flake8` concepts, `pycodestyle`
- **Auto-formatting**: `black`, `ruff format`
- **Import sorting**: `isort`, `ruff`'s isort rules
- **pytest fundamentals**: Test discovery, assertions, fixtures
- **`@pytest.mark.parametrize`**: Data-driven tests
- **Fixture patterns**: Scope, `autouse`, factory fixtures
- **Protocol-based mocking** for testability
- **Test organization**: `conftest.py`, test classes, naming

## Linting: Catching Errors Before Runtime

A **linter** analyzes source code for potential errors, style violations, and suspicious
constructs without actually running the code. Modern Python projects typically use
`ruff` as a fast, unified linter that replaces `flake8`, `pycodestyle`, `pyflakes`,
and many other tools.

**Common lint categories:**
- **E/W** (pycodestyle): Style violations (whitespace, line length, etc.)
- **F** (pyflakes): Logical errors (unused imports, undefined names)
- **I** (isort): Import ordering violations
- **UP** (pyupgrade): Code that can use newer Python syntax
- **B** (bugbear): Likely bugs and design problems

In [ ]:
# Examples of common lint violations and their fixes
#
# We cannot run ruff directly in a notebook, but we can demonstrate
# the kinds of issues linters catch.

# --- E501: Line too long ---
# BAD: exceeds 79 characters
# result = some_function(first_argument, second_argument, third_argument, fourth_argument, fifth_argument)

# GOOD: break into multiple lines
# result = some_function(
#     first_argument,
#     second_argument,
#     third_argument,
# )

# --- F401: Unused import ---
# BAD:
# import os  # never used in the module

# --- F841: Local variable assigned but never used ---
# BAD:
# x = compute_something()  # x is never read

# --- E711: Comparison to None ---
# BAD:
value: str | None = "hello"
# if value == None:  # noqa: E711
#     pass

# GOOD:
if value is None:
    pass

# --- UP035: Use newer import path ---
# BAD (Python < 3.9 style):
# from typing import List, Dict

# GOOD (Python 3.9+):
# Use list[str], dict[str, int] directly

# --- B006: Mutable default argument ---
# BAD:
def bad_default(items: list[str] = []) -> list[str]:  # noqa: B006
    items.append("new")
    return items

# GOOD:
def good_default(items: list[str] | None = None) -> list[str]:
    if items is None:
        items = []
    items.append("new")
    return items


print("Mutable default argument bug:")
print(f"  bad_default() = {bad_default()}")
print(f"  bad_default() = {bad_default()}")  # Bug: returns ['new', 'new']
print(f"  bad_default() = {bad_default()}")  # Bug: returns ['new', 'new', 'new']

print(f"\n  good_default() = {good_default()}")
print(f"  good_default() = {good_default()}")  # Correct: always ['new']

In [ ]:
# ruff configuration in pyproject.toml
#
# ruff is configured via pyproject.toml (or ruff.toml).
# Here is a typical configuration:

ruff_config = """
[tool.ruff]
# Target Python version
target-version = "py312"

# Line length (matching black's default)
line-length = 88

[tool.ruff.lint]
# Enable rule categories
select = [
    "E",    # pycodestyle errors
    "W",    # pycodestyle warnings
    "F",    # pyflakes
    "I",    # isort
    "UP",   # pyupgrade
    "B",    # flake8-bugbear
    "SIM",  # flake8-simplify
    "RUF",  # ruff-specific rules
]

# Ignore specific rules
ignore = [
    "E501",  # line too long (handled by formatter)
]

[tool.ruff.lint.per-file-ignores]
# Allow unused imports in __init__.py (re-exports)
"__init__.py" = ["F401"]
# Allow assert in tests
"tests/**" = ["S101"]

[tool.ruff.lint.isort]
# Force single-line imports for cleaner diffs
force-single-line = true
"""

print("Typical ruff configuration (pyproject.toml):")
print(ruff_config)

# Command-line usage:
print("Common ruff commands:")
print("  ruff check .              # Lint all files")
print("  ruff check --fix .        # Lint and auto-fix safe issues")
print("  ruff check --select E,W . # Check only pycodestyle rules")

## Auto-Formatting: black and ruff format

Auto-formatters rewrite your code to conform to a consistent style, eliminating
debates about formatting. **black** is the most popular Python formatter, and
**ruff format** provides a near-identical output at much higher speed.

Key principles of black:
- Deterministic: same input always produces same output
- Line length: 88 characters by default
- Trailing commas trigger vertical formatting
- Double quotes for strings by default

In [ ]:
# How black/ruff format transforms code
#
# Before formatting:
before = '''
x = {  'a':37,'b':42, 'c':927}
y = 'hello ''world'
z = 'hello '+'world'
a = [1,2, 3,  4]
zzz = {k: v for k,v in y.items() if k == 'a' or k == 'b'}
def f  (a,  b,  c):
    return a+b+c
'''

# After formatting (what black/ruff format produces):
after = '''
x = {"a": 37, "b": 42, "c": 927}
y = "hello " "world"
z = "hello " + "world"
a = [1, 2, 3, 4]
zzz = {k: v for k, v in y.items() if k == "a" or k == "b"}


def f(a, b, c):
    return a + b + c
'''

print("Before auto-formatting:")
print(before)
print("After auto-formatting:")
print(after)

# Trailing comma magic: forces vertical layout
print("Trailing comma triggers vertical formatting:")
print("""  # Stays on one line (no trailing comma):""")
print("""  result = func(arg1, arg2, arg3)""")
print("""""")
print("""  # Expands vertically (trailing comma present):""")
print("""  result = func(
      arg1,
      arg2,
      arg3,  # <-- trailing comma
  )""")

# Configuration in pyproject.toml
print("\nFormatter config (pyproject.toml):")
print("  [tool.black]")
print('  line-length = 88')
print('  target-version = ["py312"]')
print("\n  # Or for ruff format:")
print("  [tool.ruff.format]")
print('  quote-style = "double"')
print('  indent-style = "space"')

In [ ]:
# Import Sorting: isort and ruff's isort rules
#
# isort (or ruff's I rules) automatically sorts and groups imports
# according to PEP 8 conventions.

# Before isort:
before_isort = """
from myapp.utils import helper
import sys
from pathlib import Path
import requests
import os
from collections import defaultdict
from flask import Flask
from myapp.models import User
"""

# After isort:
after_isort = """
import os
import sys
from collections import defaultdict
from pathlib import Path

import requests
from flask import Flask

from myapp.models import User
from myapp.utils import helper
"""

print("Before isort:")
print(before_isort)
print("After isort:")
print(after_isort)

# isort configuration compatible with black
print("isort config (pyproject.toml):")
print("  [tool.isort]")
print('  profile = "black"')
print("  force_single_line = true")
print()
print("Common commands:")
print("  isort .                 # Sort all imports")
print("  isort --check-only .    # Check without modifying")
print("  ruff check --select I . # Check imports with ruff")
print("  ruff check --fix --select I .  # Fix imports with ruff")

## pytest Fundamentals

`pytest` is the standard testing framework for Python. It provides simple assertion
syntax, powerful fixtures, parameterized tests, and a plugin ecosystem.

**Test discovery rules:**
- Test files: `test_*.py` or `*_test.py`
- Test functions: `test_*` prefix
- Test classes: `Test*` prefix (no `__init__` method)
- Test methods: `test_*` prefix inside `Test*` classes

In [ ]:
# Code under test: a simple calculator module

class Calculator:
    """A basic calculator with history tracking."""

    def __init__(self) -> None:
        self.history: list[str] = []

    def add(self, a: float, b: float) -> float:
        """Return the sum of a and b."""
        result = a + b
        self.history.append(f"{a} + {b} = {result}")
        return result

    def divide(self, a: float, b: float) -> float:
        """Return a divided by b.

        Raises:
            ZeroDivisionError: If b is zero.
        """
        if b == 0:
            raise ZeroDivisionError("Cannot divide by zero")
        result = a / b
        self.history.append(f"{a} / {b} = {result}")
        return result

    def clear_history(self) -> None:
        """Clear the calculation history."""
        self.history.clear()


# Demonstrate what pytest tests look like
# In a real project, this would be in tests/test_calculator.py

def test_add() -> None:
    """Test basic addition."""
    calc = Calculator()
    assert calc.add(2, 3) == 5
    assert calc.add(-1, 1) == 0
    assert calc.add(0.1, 0.2) - 0.3 < 1e-10  # float comparison


def test_divide() -> None:
    """Test basic division."""
    calc = Calculator()
    assert calc.divide(10, 2) == 5.0
    assert calc.divide(7, 2) == 3.5


def test_divide_by_zero() -> None:
    """Test that dividing by zero raises ZeroDivisionError."""
    calc = Calculator()
    try:
        calc.divide(1, 0)
        assert False, "Should have raised ZeroDivisionError"
    except ZeroDivisionError as e:
        assert str(e) == "Cannot divide by zero"


def test_history() -> None:
    """Test that operations are recorded in history."""
    calc = Calculator()
    calc.add(1, 2)
    calc.divide(10, 5)
    assert len(calc.history) == 2
    assert "1 + 2 = 3" in calc.history[0]


# Run the tests manually in the notebook
for test_func in [test_add, test_divide, test_divide_by_zero, test_history]:
    try:
        test_func()
        print(f"  PASSED: {test_func.__name__}")
    except AssertionError as e:
        print(f"  FAILED: {test_func.__name__} - {e}")

In [ ]:
# @pytest.mark.parametrize: Data-Driven Tests
#
# Parametrize lets you run the same test with multiple sets of inputs.
# This avoids duplicating test logic and makes it easy to add cases.

# In a real test file, you would use:
# import pytest
# @pytest.mark.parametrize("a, b, expected", [
#     (2, 3, 5),
#     (-1, 1, 0),
#     (0, 0, 0),
#     (100, 200, 300),
# ])
# def test_add(a: float, b: float, expected: float) -> None:
#     calc = Calculator()
#     assert calc.add(a, b) == expected

# Simulating parametrize behavior in the notebook
test_cases_add: list[tuple[float, float, float]] = [
    (2, 3, 5),
    (-1, 1, 0),
    (0, 0, 0),
    (100, 200, 300),
    (-5, -3, -8),
]

print("Parametrized test_add:")
calc = Calculator()
for a, b, expected in test_cases_add:
    result = calc.add(a, b)
    status = "PASSED" if result == expected else "FAILED"
    print(f"  {status}: add({a}, {b}) == {expected} (got {result})")

# Parametrize with IDs for readable output
# @pytest.mark.parametrize("input_str, expected", [
#     pytest.param("hello", "HELLO", id="simple"),
#     pytest.param("", "", id="empty"),
#     pytest.param("Hello World", "HELLO WORLD", id="mixed-case"),
# ])
# def test_upper(input_str: str, expected: str) -> None:
#     assert input_str.upper() == expected

test_cases_upper: list[tuple[str, str, str]] = [
    ("hello", "HELLO", "simple"),
    ("", "", "empty"),
    ("Hello World", "HELLO WORLD", "mixed-case"),
]

print("\nParametrized test_upper (with IDs):")
for input_str, expected, test_id in test_cases_upper:
    result = input_str.upper()
    status = "PASSED" if result == expected else "FAILED"
    print(f"  {status} [{test_id}]: {input_str!r}.upper() == {expected!r}")

In [ ]:
# Fixtures: Setup and Teardown for Tests
#
# pytest fixtures provide reusable setup/teardown logic.
# They are dependency-injected by name into test functions.

# In a real test file:
fixture_example = """
import pytest
from myapp.calculator import Calculator


# Basic fixture: creates a fresh Calculator for each test
@pytest.fixture
def calc() -> Calculator:
    return Calculator()


# Fixture with teardown (using yield)
@pytest.fixture
def calc_with_history() -> Calculator:
    calc = Calculator()
    calc.add(1, 2)
    calc.add(3, 4)
    yield calc          # Test runs here
    calc.clear_history() # Teardown runs after the test


# Fixture scopes: "function" (default), "class", "module", "session"
@pytest.fixture(scope="module")
def db_connection():
    """Created once per module, shared across all tests in the module."""
    conn = create_connection()
    yield conn
    conn.close()


# autouse: automatically applied to all tests in scope
@pytest.fixture(autouse=True)
def reset_global_state():
    """Reset global state before each test."""
    global_config.reset()
    yield


# Factory fixture: returns a factory function for flexible setup
@pytest.fixture
def make_calculator():
    """Factory that creates calculators with optional preset history."""
    created: list[Calculator] = []

    def _factory(preload: list[tuple[float, float]] | None = None) -> Calculator:
        calc = Calculator()
        if preload:
            for a, b in preload:
                calc.add(a, b)
        created.append(calc)
        return calc

    yield _factory

    # Teardown: clear all created calculators
    for c in created:
        c.clear_history()


# Using fixtures in tests
def test_add_simple(calc: Calculator) -> None:
    assert calc.add(2, 3) == 5


def test_with_preloaded_history(make_calculator) -> None:
    calc = make_calculator(preload=[(1, 2), (3, 4)])
    assert len(calc.history) == 2
    assert calc.add(5, 6) == 11
    assert len(calc.history) == 3
"""

print("pytest fixture patterns:")
print(fixture_example)

In [ ]:
# Protocol-Based Mocking for Testability
#
# Instead of patching with unittest.mock, use Protocols to define
# interfaces and inject test implementations. This makes tests
# simpler and less brittle.

from typing import Protocol


# Define the interface using Protocol
class EmailSender(Protocol):
    """Protocol for sending emails."""

    def send(self, to: str, subject: str, body: str) -> bool: ...


# Production implementation
class SmtpEmailSender:
    """Real email sender using SMTP."""

    def __init__(self, host: str, port: int) -> None:
        self.host = host
        self.port = port

    def send(self, to: str, subject: str, body: str) -> bool:
        # In production, this would actually send an email
        print(f"  [SMTP] Sending to {to}: {subject}")
        return True


# Test implementation (no mocking library needed!)
class FakeEmailSender:
    """Fake email sender for testing."""

    def __init__(self) -> None:
        self.sent: list[tuple[str, str, str]] = []

    def send(self, to: str, subject: str, body: str) -> bool:
        self.sent.append((to, subject, body))
        return True


# Business logic depends on the Protocol, not the implementation
class UserService:
    """Service that manages users and sends notifications."""

    def __init__(self, email_sender: EmailSender) -> None:
        self.email_sender = email_sender

    def register_user(self, email: str, name: str) -> dict[str, str]:
        """Register a new user and send a welcome email."""
        user = {"email": email, "name": name, "status": "active"}
        self.email_sender.send(
            to=email,
            subject="Welcome!",
            body=f"Hello {name}, welcome to our service!",
        )
        return user


# Test using the fake -- no mock.patch needed
def test_register_user_sends_welcome_email() -> None:
    """Test that registering a user sends a welcome email."""
    fake_sender = FakeEmailSender()
    service = UserService(email_sender=fake_sender)

    user = service.register_user("alice@example.com", "Alice")

    assert user["status"] == "active"
    assert len(fake_sender.sent) == 1
    to, subject, body = fake_sender.sent[0]
    assert to == "alice@example.com"
    assert subject == "Welcome!"
    assert "Alice" in body


# Run the test
test_register_user_sends_welcome_email()
print("PASSED: test_register_user_sends_welcome_email")

# Show production usage
print("\nProduction usage:")
prod_sender = SmtpEmailSender("smtp.example.com", 587)
prod_service = UserService(email_sender=prod_sender)
prod_service.register_user("bob@example.com", "Bob")

In [ ]:
# Test Organization: conftest.py, Test Classes, and Naming
#
# A well-organized test suite follows conventions that make tests
# easy to find, understand, and maintain.

project_layout = """
Project test organization:

my_project/
    src/
        my_project/
            __init__.py
            calculator.py
            user_service.py
            email/
                __init__.py
                sender.py
    tests/
        conftest.py              # Shared fixtures for all tests
        test_calculator.py       # Tests for calculator module
        test_user_service.py     # Tests for user_service module
        email/
            conftest.py          # Fixtures specific to email tests
            test_sender.py       # Tests for email sender
"""

conftest_example = '''
# tests/conftest.py
#
# conftest.py files are automatically loaded by pytest.
# Fixtures defined here are available to all tests in the
# same directory and subdirectories.

import pytest
from my_project.calculator import Calculator
from my_project.email.sender import FakeEmailSender


@pytest.fixture
def calculator() -> Calculator:
    """Provide a fresh Calculator instance."""
    return Calculator()


@pytest.fixture
def fake_email_sender() -> FakeEmailSender:
    """Provide a fake email sender for testing."""
    return FakeEmailSender()
'''

test_class_example = '''
# tests/test_calculator.py
#
# Test classes group related tests. No __init__ method needed.


class TestAdd:
    """Tests for Calculator.add()."""

    def test_positive_numbers(self, calculator: Calculator) -> None:
        assert calculator.add(2, 3) == 5

    def test_negative_numbers(self, calculator: Calculator) -> None:
        assert calculator.add(-2, -3) == -5

    def test_mixed_signs(self, calculator: Calculator) -> None:
        assert calculator.add(-1, 1) == 0


class TestDivide:
    """Tests for Calculator.divide()."""

    def test_even_division(self, calculator: Calculator) -> None:
        assert calculator.divide(10, 2) == 5.0

    def test_float_result(self, calculator: Calculator) -> None:
        assert calculator.divide(7, 2) == 3.5

    def test_divide_by_zero_raises(self, calculator: Calculator) -> None:
        with pytest.raises(ZeroDivisionError, match="Cannot divide by zero"):
            calculator.divide(1, 0)
'''

print(project_layout)
print("conftest.py:")
print(conftest_example)
print("Test classes:")
print(test_class_example)

In [ ]:
# pytest Command-Line Usage and Configuration

pytest_config = """
# pyproject.toml pytest configuration

[tool.pytest.ini_options]
# Minimum pytest version required
minversion = "7.0"

# Default command-line options
addopts = [
    "-ra",           # Show extra test summary for all except passed
    "--strict-markers",  # Error on unregistered markers
    "--tb=short",    # Shorter traceback format
]

# Test discovery paths
testpaths = ["tests"]

# Register custom markers
markers = [
    "slow: marks tests as slow (deselect with '-m not slow')",
    "integration: marks integration tests",
]
"""

print(pytest_config)

print("Common pytest commands:")
print("  pytest                       # Run all tests")
print("  pytest tests/test_calc.py    # Run specific file")
print("  pytest -k 'test_add'         # Run tests matching pattern")
print("  pytest -m 'not slow'         # Skip tests marked as slow")
print("  pytest -x                    # Stop on first failure")
print("  pytest --lf                  # Rerun only last failures")
print("  pytest -v                    # Verbose output")
print("  pytest --co                  # Collect only (show what would run)")
print("  pytest --cov=src             # Measure code coverage")

## Summary

### Key Takeaways

| Tool / Concept | Purpose | Key Command |
|---------------|---------|-------------|
| **ruff** | Fast, unified linter | `ruff check .` |
| **ruff --fix** | Auto-fix lint issues | `ruff check --fix .` |
| **black / ruff format** | Auto-format code | `black .` or `ruff format .` |
| **isort / ruff I rules** | Sort imports | `isort .` or `ruff check --select I .` |
| **pytest** | Run tests | `pytest` |
| **@pytest.mark.parametrize** | Data-driven tests | Multiple inputs, one test function |
| **Fixtures** | Reusable setup/teardown | `@pytest.fixture` |
| **Factory fixtures** | Flexible object creation | Return a factory callable |
| **Protocol mocking** | Testable dependencies | Inject fake implementations |
| **conftest.py** | Shared fixtures | Auto-loaded by pytest |

### Best Practices
- Use `ruff` as your single linting and formatting tool for simplicity
- Configure all tools in `pyproject.toml` for a single source of truth
- Write tests that are independent, deterministic, and fast
- Use `@pytest.mark.parametrize` to avoid duplicating test logic
- Prefer Protocol-based dependency injection over `mock.patch` for cleaner tests
- Organize tests to mirror your source code structure
- Use `conftest.py` for fixtures shared across multiple test files
- Run `pytest --cov` to track test coverage and find untested code